<a href="https://colab.research.google.com/github/eengel7/transformer_speech_transcription/blob/main/feature_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feature Pipeline

This notebook downloads and pre-processed the data set Common Voice.Common Voice is a series of crowd-sourced datasets where speakers record text from Wikipedia in various languages. We'll use the latest edition of the Common Voice dataset (version 11). 

Once the data is loaded, we remove meta data that is irrelevant for the training task and we utilise the following classes provided by Hugging Face Transformer:



*   **WhisperFeatureExtractor**: This class performs padding and spectrogram conversion. It first pads/truncates a batch of audio samples such that all samples have an input length of 30s. Samples shorter than 30s are padded to 30s by appending zeros to the end of the sequence (zeros in an audio signal corresponding to no signal or silence). After that, it is converting the padded audio arrays to log-Mel spectrograms. These spectrograms are a visual representation of the frequencies of a signal.

*   **WhisperTokenizer**: This class maps a sequence of text tokens to the actual text string. It decodes using Connectionist Temporal Classification (CTC) and thus we need to train a CTC tokenizer for each language and data set used.

*   ->> **WhisperProcessor**: This final class combines the two above classes, the feature extractor and tokenizer.


After having defined the above classes, we apply the data preparation function to all of our training examples using the library dataset.The resulting pre-processed data set is storedin gdrive. 




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!add-apt-repository -y ppa:jonathonf/ffmpeg-4
!apt update
!apt install -y ffmpeg
!pip install huggingface_hub
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio
!pip install huggingface
!pip install transformers

from IPython.display import clear_output 
clear_output()

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [ ]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "sv-SE", split="train+validation", use_auth_token=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "sv-SE", split="test", use_auth_token=True)


Extracting data files #3:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #4:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #2:   0%|          | 0/1 [00:00<?, ?obj/s]

...

Generating other split: 0 examples [00:00, ? examples/s]





Reading metadata...: 0it [00:00, ?it/s]



Reading metadata...: 5699it [00:00, 46560.89it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]






Reading metadata...: 1346it [00:00, 59425.41it/s]


Dataset common_voice_11_0 downloaded and prepared to /root/.cache/huggingface/datasets/mozilla-foundation___common_voice_11_0/sv-SE/11.0.0/f8e47235d9b4e68fa24ed71d63266a02018ccf7194b2a8c9c598a5f3ab304d9f. Subsequent calls will reuse this data.


In [ ]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

In [ ]:
common_voice.save_to_disk("drive/MyDrive/CommonVoice/common_voice_swedish.hf")

In [ ]:
from transformers import WhisperFeatureExtractor, WhisperTokenizer

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")


tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Swedish", task="transcribe")



Downloading:   0%|          | 0.00/185k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/829 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/494k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

In [ ]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))


In [ ]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

# , remove_columns=common_voice.column_names["test"]
common_voice = common_voice.map(prepare_dataset, num_proc=2)

#1:   0%|          | 0/6180 [00:00<?, ?ex/s]

#0:   0%|          | 0/6180 [00:00<?, ?ex/s]

#1:   0%|          | 0/2534 [00:00<?, ?ex/s]

#0:   0%|          | 0/2535 [00:00<?, ?ex/s]

In [ ]:
common_voice.save_to_disk("drive/MyDrive/CommonVoice/common_voice_swedish_preprocessed.hf")